In [ ]:

import chi, os, time
from chi import server, context
from chi import network # Still needed for floating IP

PROJECT_NUMBER = 36 
SHARED_KEY_NAME = "project36_key"
# --- ===------------------------------------=== ---


try:
    print("Attempting to select project...")
    context.choose_project()
    
    print("Project context selection attempted.")
except Exception as e:
    print(f"Error during project selection: {e}")
    print("Please ensure you have access to at least one project.")
    assert False, "Project selection failed."

try:
    print("Attempting to select site (defaulting to KVM@TACC)...")
    context.choose_site(default="KVM@TACC")
    print("Site context selection attempted (KVM@TACC as default).")
except Exception as e:
     print(f"Error setting site context: {e}")
     print("Attempting interactive site choice...")
     try:
         context.choose_site() # Fallback to interactive choice
         print("Site selected interactively.")
     except Exception as e2:
         print(f"Interactive site selection failed: {e2}")
         assert False, "Site selection failed."

print(f"Proceeding with configuration. Will use key pair named: '{SHARED_KEY_NAME}'")
print("NOTE: Key pair existence check was skipped due to library errors.")

Attempting to select project...


Project context selection attempted.
Attempting to select site (defaulting to KVM@TACC)...


Site context selection attempted (KVM@TACC as default).
Proceeding with configuration. Will use key pair named: 'project36_key'
NOTE: Key pair existence check was skipped due to library errors.


In [ ]:
# VM Configuration
VM_NAME = f"legalsearch-dummy-vm-for-modelserving-project{PROJECT_NUMBER}" # Adheres to naming convention
IMAGE_NAME = "CC-Ubuntu22.04" # Standard Ubuntu 22.04 image
FLAVOR_NAME = "m1.large" 

print(f"VM Configuration:")
print(f"  Name: {VM_NAME}")
print(f"  Image: {IMAGE_NAME}")
print(f"  Flavor: {FLAVOR_NAME}")
print(f"  Key Pair: {SHARED_KEY_NAME}")

VM Configuration:
  Name: legalsearch-dummy-vm-for-modelserving-project36
  Image: CC-Ubuntu22.04
  Flavor: m1.large
  Key Pair: project36_key


In [ ]:

print(f"Initializing server object for: {VM_NAME}")
s = server.Server(
    VM_NAME,
    image_name=IMAGE_NAME,
    flavor_name=FLAVOR_NAME,
    key_name=SHARED_KEY_NAME
)

print(f"Submitting server request (idempotent=True)...")
try:
    s.submit(idempotent=True, wait=True)
    print(f"Server {s.name} ({s.id}) is ACTIVE.")
except Exception as e:
    print(f"Error submitting/waiting for server {VM_NAME}: {e}")
    try:
        s_check = server.get_server(VM_NAME)
        if s_check:
             s = s_check 
             print(f"Retrieved existing server {s.name} ({s.id}), Status: {s.status}")
             if s.status != 'ACTIVE':
                 print("Server exists but is not ACTIVE. Manual intervention might be needed via Chameleon UI.")
                 print("Attempting to wait for ACTIVE state again...")
                 server.wait_for_active(s.id) 
                 s.refresh() 
                 print(f"Server {s.name} is now {s.status}")

        else:
             print(f"Server {VM_NAME} could not be created or found.")
    except Exception as e2:
        print(f"Failed to retrieve server after error: {e2}")

if 's' not in locals() or not hasattr(s, 'id') or not s.id:
    print("\nFATAL: Server object 's' is not valid. Cannot proceed.")
    assert False, "Server object invalid, stopping notebook execution."
else:
    print(f"\nProceeding with server object for {s.name} ({s.id})")

Initializing server object for: legalsearch-dummy-vm-for-modelserving-project36
Submitting server request (idempotent=True)...
Waiting for server legalsearch-dummy-vm-for-modelserving-project36's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,legalsearch-dummy-vm-for-modelserving-project36
Id,d63eca33-562f-41fc-a628-44605f399f4e
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.0.242 (v4) Type: fixed MAC: fa:16:3e:14:72:4f
Network Name,sharednet1
Created At,2025-05-10T19:48:21Z
Keypair,project36_key
Reservation Id,None
Host Id,4f8e0835c495f995792aecd839e934f9dd1201938fdb2269b5562782


Server legalsearch-dummy-vm-for-modelserving-project36 (d63eca33-562f-41fc-a628-44605f399f4e) is ACTIVE.

Proceeding with server object for legalsearch-dummy-vm-for-modelserving-project36 (d63eca33-562f-41fc-a628-44605f399f4e)


In [ ]:

if 's' in locals() and hasattr(s, 'id') and s.id:
    try:
        print(f"Attempting to associate Floating IP with {s.name} ({s.id})...")
        s.associate_floating_ip()
        print("Association command sent. Refreshing server state...")
        # Refresh the server object to update its internal state
        s.refresh()
        print("Server state refreshed. Checking connectivity (can take up to 3 mins)...")
        # Check connectivity - this should implicitly use the associated IP now
        s.check_connectivity(timeout=180)
        print(f"Connectivity check to {s.name} successful!")

        # --- User Action Required ---
        print("\n" + "="*40)
        print("ACTION REQUIRED:")
        print("Floating IP associated successfully.")
        print("Run the NEXT cell (s.refresh() and s.show()) to view server details.")
        print("Look for the Floating IP address listed under 'Addresses' in the output.")
        print("You will need to MANUALLY COPY this IP for the SSH command.")
        print("="*40 + "\n")
        # We set a flag indicating success for the next step's printout
        floating_ip_associated_flag = True

    except Exception as e:
        print(f"\nAn error occurred during Floating IP association or connectivity check: {e}")
        print("Please check the Chameleon UI (Horizon) to see if an IP was associated manually.")
        floating_ip_associated_flag = False

else:
    print("Cannot associate Floating IP: Server object 's' not found or invalid from previous step.")
    floating_ip_associated_flag = False

# Print placeholder SSH command - USER MUST REPLACE <VM_FLOATING_IP>
print("\n" + "="*40)
print(f"SSH Command Template (Replace <VM_FLOATING_IP> manually):")
print(f"ssh -i ~/.ssh/project{PROJECT_NUMBER}_key cc@<VM_FLOATING_IP>")
print("="*40 + "\n")

if not floating_ip_associated_flag:
    print("WARNING: Floating IP association or connectivity check failed. SSH might not work.")

Attempting to associate Floating IP with legalsearch-dummy-vm-for-modelserving-project36 (d63eca33-562f-41fc-a628-44605f399f4e)...
Association command sent. Refreshing server state...
Server state refreshed. Checking connectivity (can take up to 3 mins)...
Checking connectivity to 129.114.24.228 port 22.


Connection successful
Connectivity check to legalsearch-dummy-vm-for-modelserving-project36 successful!

ACTION REQUIRED:
Floating IP associated successfully.
Run the NEXT cell (s.refresh() and s.show()) to view server details.
Look for the Floating IP address listed under 'Addresses' in the output.
You will need to MANUALLY COPY this IP for the SSH command.


SSH Command Template (Replace <VM_FLOATING_IP> manually):
ssh -i ~/.ssh/project36_key cc@<VM_FLOATING_IP>



In [ ]:


# Check if the variable 's' exists and represents a valid server object
if 's' in locals() and hasattr(s, 'id') and s.id:
    # Define the security groups needed
    security_groups_to_ensure = [
      {'name': f"allow-ssh-project{PROJECT_NUMBER}", 'port': 22, 'description': f"Enable SSH traffic (Project {PROJECT_NUMBER})"},
      {'name': f"allow-8000-fastapi-project{PROJECT_NUMBER}", 'port': 8000, 'description': f"Enable FastAPI TCP port 8000 (Project {PROJECT_NUMBER})"},
    ]

    print(f"Ensuring security groups for VM: {s.name} ({s.id})")
    os_conn = chi.clients.connection()
    try:
        nova_server = chi.nova().servers.get(s.id)
    except Exception as e:
        print(f"Error: Could not get Nova server object for ID {s.id}: {e}")
        nova_server = None

    if nova_server:
        try:
            print("Attempting to attach 'default' security group...")
            nova_server.add_security_group("default")
            print("Ensured 'default' security group is attached.")
        except Exception as e:
             err_str = str(e).lower()
             # Check for specific 400 Duplicate error or other common 'already present' variations
             if ("http 400" in err_str and "duplicate" in err_str) or \
                "already present" in err_str or \
                "is already a member" in err_str or \
                ("security group" in err_str and "is already associated" in err_str):
                 print("'default' group was already attached or add attempt failed benignly.")
             else:
                 print(f"Warning: Could not ensure default security group attachment: {e}")

        for sg_info in security_groups_to_ensure:
            sg_name = sg_info['name']
            try:
                sg = os_conn.get_security_group(sg_name)
                if not sg:
                    print(f"Creating security group: {sg_name}")
                    sg = os_conn.create_security_group(name=sg_name, description=sg_info['description'])
                    if not sg:
                         print(f"Error: Failed to get group object immediately after creation for {sg_name}")

                else:
                    print(f"Security group '{sg_name}' already exists.")

                
                try:
                    print(f"Ensuring rule for port {sg_info['port']} in {sg_name}...")
                    os_conn.create_security_group_rule(
                        sg_name, # Use group NAME here
                        port_range_min=sg_info['port'],
                        port_range_max=sg_info['port'],
                        protocol='tcp',
                        direction='ingress',
                        ethertype='IPv4',
                        remote_ip_prefix='0.0.0.0/0'
                    )
                    print(f"Rule for port {sg_info['port']} ensured.")
                except Exception as e_rule:
                    err_str_rule = str(e_rule).lower()
                    if "security group rule already exists" in err_str_rule or "duplicate rule exists" in err_str_rule:
                        print(f"Rule for port {sg_info['port']} already exists.")
                    else:
                        print(f"Warning: Checking/creating rule for port {sg_info['port']} failed: {e_rule}")


                try:
                    print(f"Attaching security group '{sg_name}' to {s.name}")
                    nova_server.add_security_group(sg_name) # Use name for nova add
                    print(f"Successfully ensured group '{sg_name}' is attached.")
                except Exception as e_add:
                     err_str_add = str(e_add).lower()
                     if "already present" in err_str_add or "is already a member" in err_str_add or ("security group" in err_str_add and "is already associated" in err_str_add):
                         print(f"Group '{sg_name}' was already attached.")
                     else:
                          print(f"Error attaching security group {sg_name}: {e_add}")

            except Exception as e_outer:
                 print(f"Outer error managing security group {sg_name}: {e_outer}")


        time.sleep(2)
        try:
            print("\nAttempting to list final attached security groups...")
            nova_server = chi.nova().servers.get(s.id)
            final_groups = nova_server.list_security_group()
            if isinstance(final_groups, list) and all(isinstance(g, dict) for g in final_groups):
                 print(f"Final security groups attached to {s.name}: {[g.get('name', 'N/A') for g in final_groups]}")
            else:
                 print(f"Unexpected format returned by list_security_group: {type(final_groups)}")
        except AttributeError:
             print("Error: nova_server.list_security_group() method not found. Skipping list.")
        except Exception as e_list:
            print(f"Could not list final security groups: {e_list}")

        print("\nSecurity group configuration finished.")

    else:
        print("Cannot configure security groups: Failed to get Nova server object.")

else:
    print("Cannot configure security groups: Server object 's' not found or invalid.")

Ensuring security groups for VM: legalsearch-dummy-vm-for-modelserving-project36 (d63eca33-562f-41fc-a628-44605f399f4e)
Attempting to attach 'default' security group...
'default' group was already attached or add attempt failed benignly.
Security group 'allow-ssh-project36' already exists.
Ensuring rule for port 22 in allow-ssh-project36...
Rule for port 22 already exists.
Attaching security group 'allow-ssh-project36' to legalsearch-dummy-vm-for-modelserving-project36
Successfully ensured group 'allow-ssh-project36' is attached.
Security group 'allow-8000-fastapi-project36' already exists.
Ensuring rule for port 8000 in allow-8000-fastapi-project36...
Rule for port 8000 already exists.
Attaching security group 'allow-8000-fastapi-project36' to legalsearch-dummy-vm-for-modelserving-project36
Successfully ensured group 'allow-8000-fastapi-project36' is attached.

Attempting to list final attached security groups...
Unexpected format returned by list_security_group: <class 'novaclient.ba

In [ ]:
if 's' in locals() and hasattr(s, 'id') and s.id:
    monitoring_ports_to_ensure = [
      {'name': f"allow-9090-prometheus-project{PROJECT_NUMBER}", 'port': 9090, 'description': f"Enable Prometheus TCP port 9090 (Project {PROJECT_NUMBER})"},
      {'name': f"allow-3000-grafana-project{PROJECT_NUMBER}", 'port': 3000, 'description': f"Enable Grafana TCP port 3000 (Project {PROJECT_NUMBER})"},
    ]

    print(f"Ensuring Monitoring security groups for VM: {s.name} ({s.id})")
    os_conn = chi.clients.connection()
    try:
        nova_server = chi.nova().servers.get(s.id)
    except Exception as e:
        print(f"Error: Could not get Nova server object for ID {s.id}: {e}")
        nova_server = None

    if nova_server:
        for sg_info in monitoring_ports_to_ensure:
            sg_name = sg_info['name']
            try:
                sg = os_conn.get_security_group(sg_name)
                if not sg:
                    print(f"Creating security group: {sg_name}")
                    sg = os_conn.create_security_group(name=sg_name, description=sg_info['description'])
                    if not sg:
                         print(f"Error: Failed to get group object immediately after creation for {sg_name}")
                         continue
                else:
                    print(f"Security group '{sg_name}' already exists.")

                try:
                    print(f"Ensuring rule for port {sg_info['port']} in {sg_name}...")
                    os_conn.create_security_group_rule(
                        sg_name, 
                        port_range_min=sg_info['port'],
                        port_range_max=sg_info['port'], protocol='tcp', direction='ingress',
                        ethertype='IPv4', remote_ip_prefix='0.0.0.0/0'
                    )
                    print(f"Rule for port {sg_info['port']} ensured.")
                except Exception as e_rule:
                    err_str_rule = str(e_rule).lower()
                    if "security group rule already exists" in err_str_rule or "duplicate rule exists" in err_str_rule:
                        print(f"Rule for port {sg_info['port']} already exists.")
                    else: print(f"Warning: Checking/creating rule for port {sg_info['port']} failed: {e_rule}")

                try:
                    print(f"Attaching security group '{sg_name}' to {s.name}")
                    nova_server.add_security_group(sg_name)
                    print(f"Successfully ensured group '{sg_name}' is attached.")
                except Exception as e_add:
                     err_str_add = str(e_add).lower()
                     if "already present" in err_str_add or "is already a member" in err_str_add or ("security group" in err_str_add and "is already associated" in err_str_add):
                         print(f"Group '{sg_name}' was already attached.")
                     else: print(f"Error attaching security group {sg_name}: {e_add}")

            except Exception as e_outer:
                 print(f"Outer error managing security group {sg_name}: {e_outer}")

        print("\nMonitoring security group configuration finished.")
      
    else: print("Cannot configure security groups: Failed to get Nova server object.")
else: print("Cannot configure security groups: Server object 's' not found or invalid.")

Ensuring Monitoring security groups for VM: legalsearch-dummy-vm-for-modelserving-project36 (d63eca33-562f-41fc-a628-44605f399f4e)
Security group 'allow-9090-prometheus-project36' already exists.
Ensuring rule for port 9090 in allow-9090-prometheus-project36...
Rule for port 9090 already exists.
Attaching security group 'allow-9090-prometheus-project36' to legalsearch-dummy-vm-for-modelserving-project36
Successfully ensured group 'allow-9090-prometheus-project36' is attached.
Security group 'allow-3000-grafana-project36' already exists.
Ensuring rule for port 3000 in allow-3000-grafana-project36...
Rule for port 3000 already exists.
Attaching security group 'allow-3000-grafana-project36' to legalsearch-dummy-vm-for-modelserving-project36
Successfully ensured group 'allow-3000-grafana-project36' is attached.

Monitoring security group configuration finished.


In [15]:
#AFTER EVERYTHING IS DONE, NOW WE SHUT IT ALL DOWN

In [16]:
# CELL 9: Terminate the VM

# Check if 's' exists and is valid
if 's' in locals() and hasattr(s, 'id') and s.id:
    # Verify the object still points to the correct server if kernel restarted
    try:
        # Refresh just in case
        s.refresh()
        vm_id_to_delete = s.id
        vm_name_to_delete = s.name
        print(f"Terminating VM: {vm_name_to_delete} ({vm_id_to_delete})...")
        try:
            s.delete() # Use the object's delete method
            print(f"Deletion request sent for {vm_name_to_delete}.")
            # Clear the variable to prevent accidental reuse
            del s
        except Exception as e:
            print(f"Error deleting server {vm_name_to_delete}: {e}")
            print("You may need to delete it manually via the Chameleon Web UI.")

    except Exception as e_refresh:
        print(f"Could not refresh server object 's', trying to find by name: {VM_NAME}")
        # Fallback if 's' object is stale - requires VM_NAME to be defined correctly
        if 'VM_NAME' in locals():
             try:
                 instance_to_delete = server.get_server(VM_NAME)
                 if instance_to_delete:
                     print(f"Found VM by name. Terminating {VM_NAME} ({instance_to_delete.id})...")
                     server.delete_server(instance_to_delete.id)
                     print(f"Deletion request sent for {VM_NAME}.")
                 else:
                     print(f"Could not find server named {VM_NAME} to delete.")
             except Exception as e_get_del:
                  print(f"Error finding/deleting server by name {VM_NAME}: {e_get_del}")
        else:
             print("VM_NAME variable not defined, cannot fallback to delete by name.")

else:
    print("Server object 's' not found or invalid. Cannot terminate VM via script.")
    print("Please terminate the VM manually via the Chameleon Web UI if needed.")

Terminating VM: legalsearch-dummy-vm-for-modelserving-project36 (933b74c6-c0d8-41ef-a433-8e5a5d615ee2)...
Deletion request sent for legalsearch-dummy-vm-for-modelserving-project36.
